# Combine Results of BHI and Zonda Models

In [29]:
import pandas as pd

bhi_df = pd.read_csv(f'../outputs/home_prices_bhi.csv')
print(bhi_df.shape)
bhi_df.head()

(34693, 13)


,Brand,Plan Name,City,State,Zip,Base Sq Ft,Bedrooms,Baths,Garage,Stories,Base Price,Predicted Price,% Difference
0,ABD Development,Courtyard 50,Davenport,FL,33837.0,2530.0,4.0,3.0,2.0,2.0,594000.0,605338.0,1.890710
1,ABD Development,Marbella 3BR,Davenport,FL,33837.0,1904.0,3.0,2.0,2.0,1.0,475000.0,542133.0,13.200402
2,ABD Development,Marbella 4BR,Davenport,FL,33837.0,2192.0,4.0,3.0,2.0,1.0,536000.0,589541.0,9.513898
3,ABD Development,Sienna,Davenport,FL,33837.0,2293.0,4.0,3.0,2.0,1.0,650000.0,605946.0,-7.015249
4,ABD Development,The Bristol,Palm Coast,FL,32137.0,5021.0,4.0,4.0,3.0,2.0,1110000.0,1088125.0,-1.990367


In [30]:
zonda_df = pd.read_csv(f'../outputs/home_prices_zonda.csv')
print(zonda_df.shape)
zonda_df.head()

(243069, 17)


,Brand,Plan Name,City,State,Zip,Latitude,Longitude,Base Sq Ft,Bedrooms,Baths,Garage,Stories,Lot Size,Sales Rate,Base Price,Predicted Price,% Difference
0,1034 NE 72nd Street LLC,Plan 1225,Seattle,WA,98115,47.681056,-122.315907,1225.0,2.0,2.0,0.0,3.0,1016.0,1.7,749900.0,782103.0,4.204023
1,1034 NE 72nd Street LLC,Plan 1643,Seattle,WA,98115,47.681056,-122.315907,1643.0,3.0,2.5,0.0,3.0,1016.0,1.7,989900.0,983704.0,-0.627912
2,1034 NE 72nd Street LLC,Plan 1737,Seattle,WA,98115,47.681056,-122.315907,1737.0,3.0,2.5,0.0,3.0,1016.0,1.7,979900.0,1028449.0,4.834754
3,13th Floor Homes,Amelia,Tamarac,FL,33319,26.192302,-80.211595,1580.0,3.0,2.5,1.0,2.0,3000.0,4.3,332000.0,350371.0,5.384523
4,13th Floor Homes,Amelia,West Palm Beach,FL,33404,26.781443,-80.082896,1558.0,3.0,2.5,1.0,2.0,2900.0,3.7,312990.0,307142.0,-1.885908


In [36]:
df = pd.merge(zonda_df, bhi_df[['Brand', 'Plan Name', 'City', 'State', 'Zip', 'Base Price', 'Predicted Price']]
    .rename(columns={'Base Price': 'Base Price BHI', 'Predicted Price': 'Predicted Price BHI'}),
    on=['Brand', 'Plan Name', 'City', 'State', 'Zip'], how='left')
df = df.rename(columns={'Base Price': 'Base Price Zonda', 'Predicted Price': 'Predicted Price Zonda'})
df['Base Price'] = (df['Base Price BHI'] + df['Base Price Zonda']) / 2
df['Predicted Price'] = (df['Predicted Price BHI'] + df['Predicted Price Zonda']) / 2
df['% Difference'] = (df['Predicted Price'] - df['Base Price']) / ((df['Predicted Price'] + df['Base Price']) / 2) * 100
df = df.drop(['Base Price', 'Predicted Price BHI', 'Predicted Price Zonda'], axis=1)
df = df.reindex(columns=['Brand', 'Plan Name', 'City', 'State', 'Zip', 'Latitude', 'Longitude', 'Base Sq Ft', 'Bedrooms', 'Baths', 'Garage', 'Stories', 'Base Price BHI', 'Base Price Zonda', 'Predicted Price', '% Difference'])
df.to_csv('../outputs/home_prices.csv', index=False)
matched_df = df[df['Base Price BHI'].notnull() & (df['Base Price BHI'] != df['Base Price Zonda'])]
print(matched_df.shape)
matched_df.sample(10)

(6885, 16)


,Brand,Plan Name,City,State,Zip,Latitude,Longitude,Base Sq Ft,Bedrooms,Baths,Garage,Stories,Base Price BHI,Base Price Zonda,Predicted Price,% Difference
219388,Taylor Morrison,Garnet,Cypress,TX,77433,29.925273,-95.726143,2684.5,3.75,3.0,3.0,1.0,541990.0,458990.0,464900.5,-7.373079
45095,D.R. Horton,DALTON,Green Cove Springs,FL,32043,30.045087,-81.755307,1986.0,4.00,2.0,2.0,1.0,346990.0,337990.0,336451.0,-1.778947
14083,Bella Vista Homes,The Sterling,Cibolo,TX,78108,29.580472,-98.205347,1966.0,3.00,2.0,2.0,1.0,372990.0,320995.0,354813.5,2.228821
99012,Highland Homes,Plan 262,Prosper,TX,75078,33.244105,-96.794786,3055.0,4.00,3.0,3.0,1.0,1091990.0,589967.5,751216.5,-11.275282
134237,Lennar,Fairmont,Bargersville,IN,46106,39.529722,-86.171950,3005.0,5.00,3.5,2.0,2.0,468995.0,466995.0,451599.5,-3.565811
132534,Lennar,Clover II,Conroe,TX,77301,30.290874,-95.417772,1932.0,4.00,2.0,2.0,1.0,310990.0,307990.0,274704.0,-11.909058
134708,Lennar,Garnet,McKinney,TX,75071,33.275736,-96.641334,2177.0,4.00,3.0,2.0,1.0,490499.0,488499.0,485427.0,-0.835345
91325,Grand Homes,Provence II,Grand Prairie,TX,75054,32.577016,-97.047807,3472.0,3.00,2.5,2.0,2.0,662900.0,682900.0,560830.5,-18.167582
127877,Landsea Homes,Newcastle,Minneola,FL,34715,28.582873,-81.731522,2560.0,4.00,2.5,2.0,2.0,493999.0,489999.0,472494.0,-4.044612
190944,Ravenna Homes,Highlands 3070,Porter,TX,77365,30.134878,-95.314188,3070.0,4.00,3.0,2.0,2.0,520900.0,550900.0,507212.5,-5.500365
